# **<span style="font-family: 'Palatino Linotype', serif;">🛑✨🎤Stop right now, thank you very much</span>**
----
*<span style="font-family: 'Angilla Tattoo'"> ""Quando o amor travou o sistema, Maga Patolina, Maga Butterfly e as Spicey Girls enfrentaram o Firewall Fantasma — armadas com feitiços de estilo e coragem sincera, para encontrar o Códice do Groove Supremo e reativar o coração do Mainframe."" 🦆🦋🌶️</span>*



<div align="center">
    <img src = "Magas spice girls.png" alt = "Magas pop" width = 400>
</div>

----
 **Objetivo:** Nesse notebook, produzido pelas alunas Eloísa Maria Amador Souza e Giovana Martins Coelho, iremos implementar uma estratégia de parada antecipada em uma rede neural feita em sala de aula utilizando a biblioteca Pytorch. 

 **O que é a estratégia de parada antecipada?** A estratégia de parada antecipada calcula a precisão de classifiação ao final de cada época com os dados de validação, quando a precisão da classificação começa a piorar, o treinamento da rede neural termina.

---

In [569]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import copy
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler

In [570]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()
        
        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.ReLU(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.ReLU(),
            nn.Linear(neuronios_c2, num_targets),
        )
        
    def forward(self, b):
        b = self.camadas(b)
        return b
    
# print(b)

In [571]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1
SEMENTE_ALEATORIA = 343

In [572]:
df_pinguins = sns.load_dataset("penguins")
ATRIBUTOS = ["bill_length_mm", "bill_depth_mm", "body_mass_g"]
TARGET = ["flipper_length_mm"]

print(df_pinguins)


    species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0    Adelie  Torgersen            39.1           18.7              181.0   
1    Adelie  Torgersen            39.5           17.4              186.0   
2    Adelie  Torgersen            40.3           18.0              195.0   
3    Adelie  Torgersen             NaN            NaN                NaN   
4    Adelie  Torgersen            36.7           19.3              193.0   
..      ...        ...             ...            ...                ...   
339  Gentoo     Biscoe             NaN            NaN                NaN   
340  Gentoo     Biscoe            46.8           14.3              215.0   
341  Gentoo     Biscoe            50.4           15.7              222.0   
342  Gentoo     Biscoe            45.2           14.8              212.0   
343  Gentoo     Biscoe            49.9           16.1              213.0   

     body_mass_g     sex  
0         3750.0    Male  
1         3800.0  Female  
2     

In [573]:
df_pinguins = df_pinguins.dropna(axis=0)

In [574]:
indices = df_pinguins.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df_pinguins.loc[indices_treino_val]
df_teste = df_pinguins.loc[indices_teste]

X_teste = df_teste.reindex(ATRIBUTOS, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [575]:
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_pinguins.loc[indices_treino]
df_val = df_pinguins.loc[indices_val]

X_treino = df_treino.reindex(ATRIBUTOS, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values

X_val = df_val.reindex(ATRIBUTOS, axis=1).values
y_val = df_val.reindex(TARGET, axis=1).values

In [576]:
x_scaler = MaxAbsScaler()
x_scaler.fit(X_treino)

y_scaler = MaxAbsScaler()
y_scaler.fit(y_treino)

X_treino = x_scaler.transform(X_treino)
y_treino = y_scaler.transform(y_treino)

X_val = x_scaler.transform(X_val)
y_val = y_scaler.transform(y_val)

X_teste = x_scaler.transform(X_teste)
y_teste = y_scaler.transform(y_teste)

In [577]:
X_teste = torch.tensor(X_teste).float()
X_treino = torch.tensor(X_treino).float()
X_val = torch.tensor(X_val).float()
Y_teste = torch.tensor(y_teste).float()
Y_treino = torch.tensor(y_treino).float()
Y_val = torch.tensor(y_val).float()

In [578]:
NUM_DADOS_DE_ENTRADA = len(ATRIBUTOS)
NUM_DADOS_DE_SAIDA = len(TARGET) 
NEURONIOS_C1 = 10
NEURONIOS_C2 = 8

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

In [579]:
TAXA_DE_APRENDIZADO = 0.45

otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)

In [580]:
fn_perda = nn.MSELoss()

In [581]:
def score_function(engine):
    val_loss = engine.state.metrics['val_loss']
    return -val_loss

In [582]:
NUM_EPOCAS = 500
melhor_perda  = float('inf')
melhor_peso = None
paciencia = 5

for epoca in range(NUM_EPOCAS):
    minha_mlp.train()

    # forward pass
    y_pred = minha_mlp(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(Y_treino, y_pred)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    minha_mlp.eval()
    with torch.no_grad():
        valor_output = minha_mlp(X_val)
        valor_perda = fn_perda(valor_output, Y_val.float())  
        # y_plot.append(valor_perda.item())
        # x_plot.append(epoca)

    if valor_perda.item() < melhor_perda:
        melhor_perda = valor_perda.item()
        melhor_peso = copy.deepcopy(minha_mlp.state_dict())
        paciencia = 5
    else:
        paciencia -= 1
        if paciencia == 0:
            break


    print(epoca, melhor_perda)  

0 0.052927467972040176
1 0.03494558483362198
2 0.007544109132140875
3 0.005751939956098795
4 0.0032805735245347023
5 0.0032805735245347023
6 0.0030458499677479267
7 0.0030458499677479267
8 0.003030559280887246
9 0.003030559280887246
10 0.0029961285181343555
11 0.0029784839134663343
12 0.002940878737717867
13 0.002913328120484948
14 0.0028797141276299953
15 0.0028492645360529423
16 0.002817192580550909
17 0.002786391880363226
18 0.002754362067207694
19 0.0027206684462726116
20 0.002683988306671381
21 0.00264498102478683
22 0.0026059008669108152
23 0.0025666134897619486
24 0.0025271750055253506
25 0.002487744437530637
26 0.002448413986712694
27 0.002409011824056506
28 0.002369929337874055
29 0.0023310657124966383
30 0.0022923091892153025
31 0.002253703773021698
32 0.0022152739111334085
33 0.002177175600081682
34 0.002142141107469797
35 0.002109046094119549
36 0.002077271696180105
37 0.0020458886865526438
38 0.002014933852478862
39 0.0019842716865241528
40 0.0019538807682693005
41 0.00192

224 0.0006658780621364713
225 0.0006653114687651396
226 0.000664748135022819
227 0.0006642340449616313
228 0.0006638052291236818
229 0.0006634107558056712
230 0.0006630001589655876
231 0.000662623904645443
232 0.0006622336804866791
233 0.0006618749466724694
234 0.0006615036982111633
235 0.0006611653952859342
236 0.0006608086405321956
237 0.0006604854133911431
238 0.0006601496716029942
239 0.0006598280742764473
240 0.0006595214363187551
241 0.0006592075806111097
242 0.0006589136319234967
243 0.0006586136878468096
244 0.000658333592582494
245 0.0006580472690984607
246 0.0006576764862984419
247 0.0006573821301572025
248 0.0006570104160346091
249 0.000656727934256196
250 0.0006563701899722219
251 0.0006560978363268077
252 0.000655754585750401
253 0.0006554920109920204
254 0.00065516063477844
255 0.0006549077806994319
256 0.0006545901414938271
257 0.0006543454364873469
258 0.000654039264190942
259 0.0006538036977872252
260 0.0006535163847729564
261 0.0006532917032018304
262 0.00065301469294

In [583]:
y_prev = minha_mlp(X_teste)
y_prev

tensor([[0.7977],
        [0.9500],
        [0.9635],
        [0.8379],
        [0.9422],
        [0.9539],
        [0.8206],
        [0.8610],
        [0.9240],
        [0.9431],
        [0.9185],
        [0.8185],
        [0.8505],
        [0.8266],
        [0.9393],
        [0.8528],
        [0.8312],
        [0.8312],
        [0.8177],
        [0.9558],
        [0.9275],
        [0.8543],
        [0.7947],
        [0.8158],
        [0.8081],
        [0.8078],
        [0.8254],
        [0.8585],
        [0.8276],
        [0.8734],
        [0.8605],
        [0.8329],
        [0.8357],
        [0.8048]], grad_fn=<AddmmBackward0>)

-----

### 📊 Conclusão:

Nesse notebook, exploramos a ferramentas de *Early stopping*, que é uma estratégia útil para evitar o *overfitting* das redes neurais. Isso é importante pois nos ajuda a obter uma previsão adequada dos dados, além de diminuir o custo computacional do treinamento da rede, tornando o processo mais eficiente.

----
### 📚 Referências:

1. SOUZA, Vinícius. Usando Early Stopping para definir o número de épocas de treinamento. 2021. Disponível em: <https://www.deeplearningbook.com.br/usando-early-stopping-para-definir-o-numero-de-epocas-de-treinamento/>. Acesso em: 13 abr. 2025.

2. USER12717497. Early stopping in PyTorch. Stack Overflow, 6 abr. 2022. Disponível em: <https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch>. Acesso em: 13 abr. 2025.

3. BHATT, Vrunda. A step-by-step guide to early stopping in TensorFlow and PyTorch. Medium, 4 jul. 2023. Disponível em: <https://medium.com/@vrunda.bhattbhatt/a-step-by-step-guide-to-early-stopping-in-tensorflow-and-pytorch-59c1e3d0e376>. Acesso em: 13 abr. 2025.

4. OPENAI. ChatGPT. 2025. Disponível em: <https://chatgpt.com/share/67fbf75f-8500-8005-8251-84493623cf41>. Acesso em: 13 abr. 2025.